# **ME 193, Spring 2021**
## Final Project Notebook 


**Notes**:

1. 

In [1]:
import os
import numpy as np
import pandas as pd
import sklearn.tree as tree
import sklearn.metrics as mt
import sklearn.ensemble as ens
import matplotlib.pyplot as plt
import sklearn.model_selection

SyntaxError: invalid syntax (<ipython-input-1-e0ed7fb0e6d3>, line 8)